<a href="https://colab.research.google.com/github/bhirwa/abtest-mlops/blob/main/A_b_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A/B Testing with Machine Learning

A/b testing compares the conversion rates of only two groups; Exposed group and the Control group. But in the real world, the situation and even the behaviours of online users is complex and more dynamic. Situations such as:

- Online users spend a different amount of time in the Advertisement.
- Users take different paths to the Ad.
- Users use different devices and Also view the Ad on different dates.

With all these complexity, machine learning helps in generating insights from these complex systems.

In [ ]:
#importing libraries for the project
import pandas as pd
'''import sklearn.utils
from matplotlib import pyplot
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from matplotlib import pyplot
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
import xgboost
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.svm import SVR
import seaborn as sns
import matplotlib.pyplot as plt'''


'import sklearn.utils\nfrom matplotlib import pyplot\nimport numpy as np\nfrom sklearn.tree import DecisionTreeRegressor\nfrom matplotlib import pyplot\nfrom sklearn.datasets import make_regression\nfrom sklearn.preprocessing import StandardScaler\nfrom keras.models import Sequential\nfrom keras.layers import Dense\nfrom keras.optimizers import SGD\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.model_selection import KFold\nfrom sklearn.model_selection import cross_val_score\nfrom sklearn.preprocessing import MinMaxScaler\nfrom sklearn.preprocessing import LabelEncoder\nfrom sklearn.tree import DecisionTreeClassifier\nimport xgboost\nfrom xgboost import XGBClassifier\nfrom sklearn.linear_model import LogisticRegression\nfrom sklearn import tree\nfrom sklearn.metrics import accuracy_score\nfrom sklearn.svm import SVR\nimport seaborn as sns\nimport matplotlib.pyplot as plt'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
path = "/content" # /content is pretty much the root. you can choose other path in your colab workspace
os.chdir(path)

In [ ]:
!ls drive


MyDrive


In [ ]:
cd drive/MyDrive

/content/drive/MyDrive


In [ ]:
# loading data using pandas
exposed_data=pd.read_csv('/content/drive/MyDrive/Exposed_group_data.csv')
control_data=pd.read_csv('/content/drive/MyDrive/control_group_data.csv')

In [ ]:

exposed_data.head()

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
3,004940f5-c642-417a-8fd2-c8e5d989f358,exposed,2020-07-04,0,Generic Smartphone,6,Chrome Mobile WebView,0,0
4,006b9c6e-5f5d-4385-a811-ff20a24b30ac,exposed,2020-07-06,8,Samsung SM-G973F,6,Chrome Mobile WebView,0,0


In [ ]:
exp_data=(exposed_data.groupby('date')
      .agg(pageviews=('yes','count'),yes=('yes','sum'),no=('no','sum'),hours=('hour','sum'))
      .reset_index()
       )

In [ ]:
exp_data


,date,pageviews,yes,no,hours
0,2020-07-03,470,43,49,3986
1,2020-07-04,477,46,45,5114
2,2020-07-05,528,35,39,5506
3,2020-07-06,294,23,25,1717
4,2020-07-07,257,22,24,2767
5,2020-07-08,714,58,61,10899
6,2020-07-09,728,55,64,8177
7,2020-07-10,538,26,42,4765


In [ ]:
control_data.head()

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0
1,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0
2,0027ce48-d3c6-4935-bb12-dfb5d5627857,control,2020-07-03,15,Samsung SM-G960F,6,Facebook,0,0
3,002e308b-1a07-49d6-8560-0fbcdcd71e4b,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0
4,00393fb9-ca32-40c0-bfcb-1bd83f319820,control,2020-07-09,5,Samsung SM-G973F,6,Facebook,0,0


In [ ]:
cont_data=(control_data.groupby('date')
      .agg(pageviews=('yes','count'),yes=('yes','sum'),no=('no','sum'),hours=('hour','sum'))
      .reset_index()
       )

In [ ]:
cont_data

,date,pageviews,yes,no,hours
0,2020-07-03,1545,104,129,23177
1,2020-07-04,426,30,38,4164
2,2020-07-05,362,17,26,3949
3,2020-07-06,196,12,14,1104
4,2020-07-07,223,16,21,2312
5,2020-07-08,484,27,31,7489
6,2020-07-09,480,30,35,5552
7,2020-07-10,355,28,28,3137



**Number summaries and basic investigations**

In [ ]:
exposed_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4006 entries, 0 to 4005
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   auction_id   4006 non-null   object
 1   experiment   4006 non-null   object
 2   date         4006 non-null   object
 3   hour         4006 non-null   int64 
 4   device_make  4006 non-null   object
 5   platform_os  4006 non-null   int64 
 6   browser      4006 non-null   object
 7   yes          4006 non-null   int64 
 8   no           4006 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 281.8+ KB


In [ ]:
control_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4071 entries, 0 to 4070
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   auction_id   4071 non-null   object
 1   experiment   4071 non-null   object
 2   date         4071 non-null   object
 3   hour         4071 non-null   int64 
 4   device_make  4071 non-null   object
 5   platform_os  4071 non-null   int64 
 6   browser      4071 non-null   object
 7   yes          4071 non-null   int64 
 8   no           4071 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 286.4+ KB


In [ ]:
exposed_data.isna().sum()

auction_id     0
experiment     0
date           0
hour           0
device_make    0
platform_os    0
browser        0
yes            0
no             0
dtype: int64

In [ ]:
control_data.isna().sum()

auction_id     0
experiment     0
date           0
hour           0
device_make    0
platform_os    0
browser        0
yes            0
no             0
dtype: int64

**Data wrangling**

In [ ]:
data_total1 = pd.concat([control_data, exposed_data])
data_total1.sample(10)


,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
3753,efdd9369-09d1-496c-8fb1-061bb366c34a,exposed,2020-07-09,3,MAR-LX1A,6,Chrome Mobile WebView,0,0
335,13e225de-5cb9-4d9b-96fb-aa137c643ce3,control,2020-07-03,15,Samsung SM-G930F,6,Facebook,1,0
1869,7643d4e5-e32b-4eb1-813e-ea8d35dcaa9d,control,2020-07-10,16,Samsung SM-A405FN,6,Samsung Internet,0,0
2857,b32ca27f-d327-469d-999e-bd85a589f224,control,2020-07-03,15,Samsung SM-G965F,6,Facebook,0,1
1432,5aa77053-9c2e-4791-a096-5d1d831c4f6d,control,2020-07-04,10,LG-$2,6,Chrome Mobile,0,0
3619,e4b7b761-2b30-4b94-9d51-49643378d593,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0
2139,8a89116b-a35f-40d0-b9c2-8fd9351df037,exposed,2020-07-04,0,Samsung SM-G920F,6,Chrome Mobile WebView,0,0
1191,4eb1365e-0a68-4ef0-9403-c81ca773970a,exposed,2020-07-05,4,iPhone,5,Chrome Mobile iOS,0,0
2313,9485c7c7-137a-4389-b472-3903cebb8511,exposed,2020-07-10,16,Generic Smartphone,6,Chrome Mobile,0,0
4,006b9c6e-5f5d-4385-a811-ff20a24b30ac,exposed,2020-07-06,8,Samsung SM-G973F,6,Chrome Mobile WebView,0,0


In [ ]:
data_total1['experiment'] = pd.factorize(data_total1.experiment)[0]

In [ ]:
from sklearn import preprocessing 
  
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
data_total1['device_make']= label_encoder.fit_transform(data_total1['device_make'])

In [ ]:
# Encode labels in column 'species'. 
data_total1['browser']= label_encoder.fit_transform(data_total1['browser'])

In [ ]:

# Encode labels in column 'species'. 
data_total1['date']= label_encoder.fit_transform(data_total1['date'])

In [ ]:
data_total1

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,00187412-2932-4542-a8ef-3633901c98d9,0,0,15,137,6,6,0,0
1,001a7785-d3fe-4e11-a344-c8735acacc2c,0,0,15,46,6,2,0,0
2,0027ce48-d3c6-4935-bb12-dfb5d5627857,0,0,15,177,6,6,0,0
3,002e308b-1a07-49d6-8560-0fbcdcd71e4b,0,0,15,46,6,2,0,0
4,00393fb9-ca32-40c0-bfcb-1bd83f319820,0,6,5,184,6,6,0,0
...,...,...,...,...,...,...,...,...,...
4001,ffbc02cb-628a-4de5-87fc-5d76b7d796e5,1,6,17,46,6,2,0,0
4002,ffc594ef-756c-4d24-a310-0d8eb4e11eb7,1,2,1,171,6,3,0,0
4003,ffdfdc09-48c7-4bfb-80f8-ec1eb633602b,1,0,4,46,6,2,0,1
4004,ffea24ec-cec1-43fb-b1d1-8f93828c2be2,1,2,7,46,6,2,0,0


In [ ]:
# Getting the features we need
data=data_total1[['hour','platform_os','date','browser','experiment','yes','no']]

In [ ]:
# Splitting the data according to features and the target feature
X1 = data.drop('yes', axis=1) #prediction features
y1 = data['yes'] #target feature